In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('mnist_train.csv')
data.head()

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
data = np.array(data)
m,n = data.shape
np.random.shuffle(data)

In [89]:
data_dev = data[:1000]
y_dev = data_dev[:,0].astype(int)
X_dev = (data_dev[:,1:]).T

data_train = data[1000:].T  # transpose this 
y_train = data_train[:,0].astype(int)
X_train = (data_train[:,1:]).T

def init_param():
    w1 = np.random.rand(10,784)-0.5
    b1 = np.random.rand(10,1)-0.5
    w2 = np.random.rand(10,10)-0.5
    b2 = np.random.rand(10,1)-0.5
    print(f"w1 shape: {w1.shape}")
    print(f"w2 shape: {w2.shape}")
    print(f"b1 shape: {b1.shape}")
    print(f"b2 shape: {b2.shape}")

    
    return w1,w2,b1,b2


def reLU(Z):
    return np.maximum(0,Z)

def softmax(Z):
    a =  np.exp(Z-np.max(Z,axis=1,keepdims=True))
    return a / np.sum(a,axis=0,keepdims=True)
    
def deriv_reLU(z):
    return z>0

def forward_prop(w1,w2,b1,b2,X):
    z1 = np.dot(w1,X) + b1
    a1 = reLU(z1)
    z2 = np.dot(w2,a1)+b2
    a2 = softmax(z2)
    return z1,z2,a1,a2

def one_hot(y):
    one_hot_y = np.zeros((y.size,y.max()+1))
    one_hot_y[np.arange(y.size),y] = 1
    one_hot_y = one_hot_y.T
    return one_hot_y



def back_prop(z1,a1,z2,a2,w1,w2,x,y,m):
    one_hot_y = one_hot(y)
    dz2 = a2-one_hot_y
    dw2 = (1/m)*(dz2.dot(a1.T))
    db2 = (1/m)(*np.sum(dz2,axis=1,keepdims=True))
    dz1 = w2.T.dot(dz2) *deriv_reLU(z1)
    dw1 = (1/m)*(dz1.dot(x.T))
    db1 = (1/m)*(np.sum(dz1,axis=1,keepdims=True))
    return dw1, db1,dw2,db2

def update_params(w1,b1,w2,b2,dw1,db1,dw2,db2,alpha):
    w1 = w1 - alpha * dw1
    b1 = b1 - alpha * db1
    w2 = w2 - alpha * dw2
    b2 = b2 - alpha * db2
    return w1,b1,w2,b2


def get_prediction(a2):
    return np.argmax(a2,0)

def get_accuracy(prediction,y):
    print(prediction,y)
    return np.sum(prediction == y) / y.size

def gradiant_descent(X,y,iteration,alpha):
    w1,b1,w2,b2 = init_param()
    #m = X.shape[1]
    for i in range(iteration):
        z1,z2,a1,a2 = forward_prop(w1,w2,b1,b2,X)
        dw1,db1,dw2,db2 = back_prop(z1,a1,z2,a2,w1,w2,X,y)
        w1,b1,w2,b2 = update_params(w1,b1,w2,b2,dw1,db1,dw2,db2,alpha)
        if (i % 10 ==0):
            print(f"Iteration: {i}")
            print(f"Accuracy: {get_accuracy(get_prediction(a2),y)}")

    return w1,b1,w2,b2

print(f"Shape of X_train: {X_train.shape}")
print(f"Shape of y_train: {y_train.shape}")
print(f"Shape of X_dev: {X_dev.shape}")
print(f"Shape of y_dev: {y_dev.shape}")


Shape of X_train: (58999, 785)
Shape of y_train: (785,)
Shape of X_dev: (784, 1000)
Shape of y_dev: (1000,)


In [90]:
w1,b1,w2,b2 = gradiant_descent(X_train,y_train,200,0.5)

w1 shape: (10, 784)
w2 shape: (10, 10)
b1 shape: (10, 1)
b2 shape: (10, 1)


ValueError: shapes (10,784) and (58999,785) not aligned: 784 (dim 1) != 58999 (dim 0)